In [1]:
import os

In [2]:
%pwd

'/home/joshua/Desktop/Refactory/SwahiliNewsClassifier/research'

In [3]:
os.chdir('../')
%pwd

'/home/joshua/Desktop/Refactory/SwahiliNewsClassifier'

In [4]:
from dataclasses import dataclass
from pathlib import Path

from swahiliNewsClassifier.constants import *
from swahiliNewsClassifier.utils.common import read_yaml, create_directories

In [5]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    train_source_URL: str
    test_source_URL: str
    train_data_file: Path
    test_data_file: Path

    unzip_dir: Path

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            train_source_URL=config.train_source_URL,
            test_source_URL=config.test_source_URL,
            train_data_file=config.train_data_file,
            test_data_file=config.test_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import zipfile
import gdown
from swahiliNewsClassifier import logger

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """
        Initialize DataIngestion object with the provided configuration.

        Args:
            config (DataIngestionConfig): Configuration object for data ingestion.
        """
        self.config = config

    def download_file(self):
        """Fetch data from a URL.
        
        Raises:
            Exception: If an error occurs during the download process.
        """
        try:
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            dataset_urls = [self.config.train_source_URL, self.config.test_source_URL]
            zip_download_dir =  [self.config.train_data_file, self.config.test_data_file]
            for index in range(2):           
            
                logger.info(f"Downloading data from {dataset_urls[index]} into file {zip_download_dir[index]}")

                file_id = dataset_urls[index].split("/")[-2]
                prefix = "https://drive.google.com/uc?/export=download&id="
                gdown.download(prefix + file_id, zip_download_dir[index])

                logger.info(f"Downloaded data from {dataset_urls[index]} into file {zip_download_dir[index]}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """Extract a zip file.

            This method extracts the contents of a zip file specified in the configuration
            to the directory specified in the configuration.

            Raises:
                Exception: If an error occurs during the extraction process.
            """
        zip_download_dir =  [self.config.train_data_file, self.config.test_data_file]
        for index in range(2):
            try:
                unzip_path = self.config.unzip_dir
                os.makedirs(unzip_path, exist_ok=True)
                
                with zipfile.ZipFile(zip_download_dir[index], "r") as zip_ref:
                    zip_ref.extractall(unzip_path)

                logger.info(f"Extracted zip file into: {unzip_path}")

            except Exception as e:
                logger.error(f"Error extracting zip file: {zip_download_dir[index]}")
                raise e
            

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-08 16:28:28,258: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-08 16:28:28,294: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-08 16:28:28,300: INFO: common: Created directory at: artifacts]
[2024-05-08 16:28:28,307: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-05-08 16:28:28,313: INFO: 4002371441: Downloading data from https://drive.google.com/file/d/15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB/view?usp=sharing into file artifacts/data_ingestion/traindata.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB
To: /home/joshua/Desktop/Refactory/SwahiliNewsClassifier/artifacts/data_ingestion/traindata.zip
100%|██████████| 3.78M/3.78M [00:01<00:00, 2.19MB/s]


[2024-05-08 16:28:35,494: INFO: 4002371441: Downloaded data from https://drive.google.com/file/d/15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB/view?usp=sharing into file artifacts/data_ingestion/traindata.zip]
[2024-05-08 16:28:35,508: INFO: 4002371441: Downloading data from https://drive.google.com/file/d/1mjmYzMdnn_UwSEgTQ7i-cJ5WSOokt9Er/view?usp=sharing into file artifacts/data_ingestion/testdata.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1mjmYzMdnn_UwSEgTQ7i-cJ5WSOokt9Er
To: /home/joshua/Desktop/Refactory/SwahiliNewsClassifier/artifacts/data_ingestion/testdata.zip
100%|██████████| 992k/992k [00:00<00:00, 1.46MB/s]

[2024-05-08 16:28:40,114: INFO: 4002371441: Downloaded data from https://drive.google.com/file/d/1mjmYzMdnn_UwSEgTQ7i-cJ5WSOokt9Er/view?usp=sharing into file artifacts/data_ingestion/testdata.zip]


[2024-05-08 16:28:40,641: INFO: 4002371441: Extracted zip file into: artifacts/data_ingestion]
[2024-05-08 16:28:40,840: INFO: 4002371441: Extracted zip file into: artifacts/data_ingestion]
